In [2]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [3]:
# Replace these with your PostgreSQL connection details
db_username = "postgres"
db_password = "postgres"
db_host = "localhost"
db_port = "5432"
db_name = "bus_trial"

# Create a database connection string
db_uri = f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create a SQLAlchemy engine
engine = create_engine(db_uri)

In [4]:
# Reading in bus static files
stops = pd.read_csv("../businfo/stops.txt", sep=",", quotechar='"')
stop_times = pd.read_csv("../businfo/stop_times.txt", sep=",", quotechar='"')
trips = pd.read_csv("../businfo/trips.txt", sep=",", quotechar='"', keep_default_na=False)
routes = pd.read_csv("../businfo/routes.txt", sep=",", quotechar='"')
shapes = pd.read_csv("../businfo/shapes.txt", sep=",", quotechar='"')

In [5]:
stop_times['stop_code'] = stop_times['stop_id'].str.extract(r'(.+)-')[0]

# Creating a DataFrame with unique pairs of route_id and shape_id
unique_pairs = trips[['route_id', 'shape_id']].drop_duplicates().sort_values(by='route_id')
shape_data = shapes.groupby('shape_id').apply(lambda group: group.sort_values('shape_pt_sequence')[['shape_pt_lon', 'shape_pt_lat']].values.tolist()).reset_index(name='location')
print(unique_pairs)

       route_id             shape_id
10276   101-202  1254-10102-fb115bad
10241   101-202  1254-10101-0364cb33
10491   105-202  1255-10502-96fc54a9
9184    105-202  1255-10501-b6191044
10714   106-202  1256-10601-5d7e5c62
...         ...                  ...
24411  WEST-201  247-810195-13525d16
24560  WEST-201  247-810201-940b3d61
24329  WEST-201  247-810146-8a99de9f
26874  WSTH-211   518-99512-09b35bfe
26858  WSTH-211   518-99511-285aec69

[574 rows x 2 columns]


In [6]:
print(shape_data)

                shape_id                                           location
0    1010-98109-d221c68a  [[174.70832, -36.54528], [174.70836, -36.54519...
1    1010-98110-34e6d81d  [[174.66655, -36.6243], [174.66656, -36.62431]...
2    1010-98111-677e70ea  [[174.69396, -36.58678], [174.69396, -36.58678...
3    1010-98112-81441893  [[174.66655, -36.6243], [174.66656, -36.62431]...
4    1010-98113-49b95bec  [[174.68896, -36.57805], [174.68822, -36.57821...
..                   ...                                                ...
569   902-98013-91fd3d3c  [[174.99147, -36.78063], [174.97737, -36.78255...
570   902-98014-6aad6783  [[174.76649, -36.84234], [174.76661, -36.84193...
571   957-12601-6c2c5b10  [[174.71207, -36.72212], [174.71207, -36.7221]...
572   957-12602-45320708  [[174.61089, -36.82247], [174.61094, -36.82244...
573   957-12603-1bf4bceb  [[174.59198, -36.75751], [174.59193, -36.75752...

[574 rows x 2 columns]


In [7]:
result_df = pd.merge(shape_data, trips, on='shape_id', how='inner')
result_df = pd.merge(result_df, routes, on='route_id', how='inner')
print(result_df[['shape_id', 'route_id']].drop_duplicates())

                  shape_id  route_id
0      1010-98109-d221c68a   981-221
36     1010-98110-34e6d81d   981-221
76     1010-98111-677e70ea   981-221
112    1010-98112-81441893   981-221
143    1010-98113-49b95bec   981-221
...                    ...       ...
27311   902-98013-91fd3d3c  MTIA-209
27316   902-98014-6aad6783  MTIA-209
27321   957-12601-6c2c5b10   126-206
27349   957-12602-45320708   126-206
27374   957-12603-1bf4bceb   126-206

[574 rows x 2 columns]


In [8]:
distinct_combinations = result_df[['shape_id', 'route_id']].drop_duplicates()

shape_id_counts = distinct_combinations['shape_id'].value_counts().reset_index()
shape_id_counts.columns = ['shape_id', 'count']


In [9]:
print(shape_id_counts)

                shape_id  count
0    1010-98109-d221c68a      1
1    1325-85604-eae716a2      1
2    1324-08302-0568b3b5      1
3    1324-08303-2dec14f4      1
4    1324-08304-bfb65991      1
..                   ...    ...
569  1175-03503-5c425428      1
570  1160-73902-8a1bf6ce      1
571  1160-73901-28fd4b64      1
572  1159-73502-42797b4e      1
573   957-12603-1bf4bceb      1

[574 rows x 2 columns]


In [10]:
print(distinct_combinations)

                  shape_id  route_id
0      1010-98109-d221c68a   981-221
36     1010-98110-34e6d81d   981-221
76     1010-98111-677e70ea   981-221
112    1010-98112-81441893   981-221
143    1010-98113-49b95bec   981-221
...                    ...       ...
27311   902-98013-91fd3d3c  MTIA-209
27316   902-98014-6aad6783  MTIA-209
27321   957-12601-6c2c5b10   126-206
27349   957-12602-45320708   126-206
27374   957-12603-1bf4bceb   126-206

[574 rows x 2 columns]


In [11]:
print(shape_data)

                shape_id                                           location
0    1010-98109-d221c68a  [[174.70832, -36.54528], [174.70836, -36.54519...
1    1010-98110-34e6d81d  [[174.66655, -36.6243], [174.66656, -36.62431]...
2    1010-98111-677e70ea  [[174.69396, -36.58678], [174.69396, -36.58678...
3    1010-98112-81441893  [[174.66655, -36.6243], [174.66656, -36.62431]...
4    1010-98113-49b95bec  [[174.68896, -36.57805], [174.68822, -36.57821...
..                   ...                                                ...
569   902-98013-91fd3d3c  [[174.99147, -36.78063], [174.97737, -36.78255...
570   902-98014-6aad6783  [[174.76649, -36.84234], [174.76661, -36.84193...
571   957-12601-6c2c5b10  [[174.71207, -36.72212], [174.71207, -36.7221]...
572   957-12602-45320708  [[174.61089, -36.82247], [174.61094, -36.82244...
573   957-12603-1bf4bceb  [[174.59198, -36.75751], [174.59193, -36.75752...

[574 rows x 2 columns]


In [12]:
# They all the same 
all(routes['route_short_name'] == routes['route_long_name'])

True

In [13]:
route_table = pd.merge(shape_data, distinct_combinations, on='shape_id', how='inner')
route_table = pd.merge(route_table, routes[["route_id", "route_short_name"]], on='route_id', how='inner')
route_table.columns = ['shape_id', 'route_coordinates', 'route_id', 'route_name']
print(route_table)

                shape_id                                  route_coordinates  \
0    1010-98109-d221c68a  [[174.70832, -36.54528], [174.70836, -36.54519...   
1    1010-98110-34e6d81d  [[174.66655, -36.6243], [174.66656, -36.62431]...   
2    1010-98111-677e70ea  [[174.69396, -36.58678], [174.69396, -36.58678...   
3    1010-98112-81441893  [[174.66655, -36.6243], [174.66656, -36.62431]...   
4    1010-98113-49b95bec  [[174.68896, -36.57805], [174.68822, -36.57821...   
..                   ...                                                ...   
569   902-98013-91fd3d3c  [[174.99147, -36.78063], [174.97737, -36.78255...   
570   902-98014-6aad6783  [[174.76649, -36.84234], [174.76661, -36.84193...   
571   957-12601-6c2c5b10  [[174.71207, -36.72212], [174.71207, -36.7221]...   
572   957-12602-45320708  [[174.61089, -36.82247], [174.61094, -36.82244...   
573   957-12603-1bf4bceb  [[174.59198, -36.75751], [174.59193, -36.75752...   

     route_id route_name  
0     981-221        981

In [15]:
subset_df = route_table[route_table['route_name'] == '70']
print(subset_df)

                shape_id                                  route_coordinates  \
166  1153-07005-b2752937  [[174.91196, -36.93226], [174.91201, -36.93223...   
167  1153-07006-016a1eb5  [[174.76994, -36.84542], [174.76994, -36.84542...   
168  1153-07007-71aa8014  [[174.80893, -36.89781], [174.80895, -36.89777...   
169  1153-07008-10bc3492  [[174.85009, -36.89845], [174.85007, -36.8985]...   
170  1153-07009-3fd13d4d  [[174.84915, -36.89832], [174.84915, -36.89832...   

    route_id route_name  
166   70-205         70  
167   70-205         70  
168   70-205         70  
169   70-205         70  
170   70-205         70  


In [95]:
import pandas as pd
from sqlalchemy import create_engine, Float, Integer, String, ARRAY

# Ensure route_coordinates is of type list
route_table['route_coordinates'] = route_table['route_coordinates'].apply(lambda x: x if isinstance(x, list) else [])

# Replace these with your PostgreSQL connection details
db_username = "postgres"
db_password = "postgres"
db_host = "localhost"
db_port = "5432"
db_name = "bus_trial"

# Create a database connection string
db_uri = f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create a SQLAlchemy engine
engine = create_engine(db_uri)

# Define the column data types
column_types = {
    'shape_id': String,
    'route_coordinates': ARRAY(Float),
    'route_id': String,
    'route_name': String
}

# Store the dataframe in the PostgreSQL database
route_table.to_sql('route_table', engine, if_exists='replace', dtype=column_types)


Exception ignored in: <function _collection_gced at 0x28fffd1f0>
Traceback (most recent call last):
  File "/Users/leonardophu/miniconda3/envs/lab/lib/python3.9/site-packages/sqlalchemy/event/registry.py", line 103, in _collection_gced
    def _collection_gced(ref: weakref.ref[Any]) -> None:
KeyboardInterrupt: 


574

In [10]:
import pandas as pd
from sqlalchemy import create_engine, Float, Integer, String, ARRAY

# Replace these with your PostgreSQL connection details
db_username = "postgres"
db_password = "postgres"
db_host = "localhost"
db_port = "5432"
db_name = "bus_trial"

# Create a database connection string
db_uri = f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create a SQLAlchemy engine
engine = create_engine(db_uri)

# Define the column data types
column_types = {
    'shape_id': String,
    'shape_pt_lat': Float,
    'shape_pt_lon': Float,
    'shape_pt_sequence': Integer
}

shapes_extract = shapes.iloc[:, [0,1,2,3]]

shapes_extract.to_sql('shapes', engine, if_exists='replace', dtype=column_types)


843